# Pfizer vaccine tweets Analysis 

![](https://www.globaltimes.cn/Portals/0/attachment/2020/2020-12-11/8c69b434-ba6c-4a1b-9574-c2c93b452e89.jpeg)

### It's been over a year since We suffered from the covid-19. And include pfizer, biontech, Many Pharmaceutical company developed a vaccine. However, people are worried that the vaccine developed in a short period of time might have side effects. Today I will analysis the Pfizer vaccine tweets and recognize their thinks

## - Import and Load datasets

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

import plotly.offline as pyo
pyo.init_notebook_mode()

import re
from wordcloud import WordCloud
import nltk
import string
from nltk.corpus import stopwords
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df =pd.read_csv("../input/pfizer-vaccine-tweets/vaccination_tweets.csv")
df.drop(["id","user_created"],axis=1,inplace=True)

df.head(5)

# 1. Tweet's counts by date

In [ ]:
px.histogram(df, x="date",nbins=100,opacity=.5,title="Tweets by date")

### Through histogram, the tweet's counts decrease after January

## How about Retweets and Favorites?

In [ ]:
px.scatter(df,x="date",y="retweets",size="retweets",hover_data=["user_name","user_location"],title="Retweets by date")

In [ ]:
px.scatter(df,x="date",y="favorites",size="favorites",hover_data=["user_name","user_location"],title="Favorites by date")

### The Favorite and Retweet counts also decrease after January.

# 2. Source (What they use?)

In [ ]:
source = pd.DataFrame(df.groupby("source")["user_name"].count())
source.reset_index(inplace=True)
source.columns=["source","count"]

fig = px.bar(source[source["count"]>5],x="source",y="count",color="source",text="count",title="Source")
fig.update_traces(textposition='outside')
fig.show()

### Most of them, use iPhone, Web and Android. TweetDeck and iPad also recorded

# 3. Tweet's city analysis

In [ ]:
df["user_location"] = df["user_location"].fillna("no_location")
def make_location(location):
    return location.split(",")[0]

df["user_location"] = df["user_location"].apply(make_location)
city = df["user_location"].value_counts()
city.drop("no_location",inplace=True)
city = pd.DataFrame(city)
city.reset_index(inplace=True)

city.columns=["city","counts"]

fig = px.bar(city.head(10), x="city", y="counts",color="city",text="counts",title="City")
fig.update_traces(textposition='outside')
fig.show()

### It has so many nan data. so use only not nan data
### The London recorded first and Other cities had similar numbers.

# 4.Tweet's text analysis!

In [ ]:
text_list = df["text"].to_list()
text = ""
for i in text_list:
    text = text + i.split("https:")[0]
    
text = text.replace(" ",",")
text = re.sub("[\@\#\n\.\…\?\\\'\d\)\(\%\*]", ",", text)
text = re.sub(",{2,}", ",", text)

text[:1000]

In [ ]:
text = text.split(',')
text[:10]

In [ ]:
def stop_w(x):
    new_s = []
    for i in text:
        if i.lower() not in stopwords.words("english"):
            new_s.append(i.lower())
    return new_s

text = stop_w(text)

plt.figure(figsize=(20,10))
FreqDist(text).plot(30)

### After remove text's url and some characters, we can get this text

### Check the text's value counts, they include so many prepositions

## WordCloud

In [ ]:
text_count = pd.Series(text).value_counts()
wc = WordCloud(width=1000, height=600, background_color="white", random_state=0)
plt.figure(figsize=(20,10),facecolor='w')
plt.imshow(wc.generate_from_frequencies(text_count))
plt.axis("off")
plt.show()

### In WordCloud, Pfizerbiontech, dose, first, vaccine, covid stick out

## Other_anlaysis...
###    If user's followers are many, they get more favorites?

In [ ]:
fig = px.scatter(df[(df["user_followers"]<2000) ], x="user_followers", y="favorites",color="favorites",range_y=(-10,300),title="Relationship of user's followers and favorites")
fig.show()

### In scatter plot, Even with a large number of followers, most received a similar number of likes as few people.